In [20]:
import pandas as pd
import datetime
from datetime import datetime as dt
import calendar

In [33]:
weekly_kbd = pd.read_excel('refinery gross inputs weekly.xls', sheet_name='Data 1', header=2)
weekly_kbd = weekly_kbd.set_index('Date')

In [38]:
weekly = weekly_kbd*7000

In [51]:
weekly.columns = weekly.columns.str.replace('Thousand Barrels per Day','bbl')

In [53]:
yearly = weekly.resample('AS-JAN').sum()

In [54]:
yearly.to_csv('yearly_inputs.csv')

In [1]:
# weekly_kbd.head()

In [26]:
yearly_2 = weekly.resample('A').sum()

In [2]:
# yearly_2.head()

In [3]:
# yearly.head()

### ---------------------- industrial gas demand UK ----------------------------------

In [5]:
nbp = pd.read_excel('gas_prices/ice_nbp.xlsx', header=1)
nbp.columns=['date', 'nbp']

In [6]:
gas_consump = pd.read_excel('gas consumption in refineries/Industrial Demand Data.xlsx', sheet_name='Sheet1', header=6)
gas_consump = gas_consump.loc[7:].copy()
gas_consump.rename(columns={'Name':'date'}, inplace=True)

In [7]:
gas_consump.date = pd.to_datetime(gas_consump.date)

In [8]:
nbp_gas = gas_consump.merge(nbp, how='left', on='date')

In [9]:
nbp_gas.set_index('date', inplace=True)

In [10]:
new_nbp = nbp_gas.astype(float)

In [11]:
nbp_gas_weekly = new_nbp.resample('w').mean()

In [63]:
# nbp_gas_weekly['2021':].to_excel('weekly_usage_data.xlsx')

In [16]:
nbp_gas_monthly = new_nbp.resample('MS').mean()
nbp_gas_monthly.reset_index(inplace=True)
nbp_gas_monthly.rename(columns={'date':'month'}, inplace=True)

In [17]:
nbp_gas_monthly.head()

month      BASF  BlackburnM   BOCTees  BPGrngmouth  BPSaltendHP  \
0 2015-04-01 -0.367667    0.000000 -0.191000    -1.089333    -0.403000   
1 2015-05-01 -0.215484    0.000000 -0.489032    -1.437097    -0.352903   
2 2015-06-01 -0.410333   -0.009333 -0.554000    -1.504333    -0.379333   
3 2015-07-01 -0.319355    0.000000 -0.548065    -1.240323    -0.442903   
4 2015-08-01 -0.283548    0.000000 -0.404194    -0.919032    -0.457097   

   BrdgewaterP   Centrax  GooleGlass  HaysChem  ICIBillnghm  ICIRuncorn  \
0     0.000000  0.000000   -0.130667 -0.003667    -1.828667   -0.053667   
1     0.000000  0.000000   -0.129677 -0.004839    -1.812258   -0.065161   
2     0.000000  0.000000   -0.131000 -0.003000    -1.713333   -0.083667   
3     0.000000 -0.005161   -0.128065 -0.002581    -1.698387   -0.074516   
4    -0.000323 -0.000323   -0.129032 -0.003548    -1.685161   -0.073548   

   Immingham  Murco  PhilipsTees  ShellStar  Winnington        nbp  
0  -4.316667    0.0        0.000  -0.313000   -0.406667  45.109500  
1  -3.252581    0.0        0.000  -0.057419   -0.440968  42.199474  
2  -3.508667    0.0       -0.003  -0.086000   -0.445000  42.649091  
3  -4.114839    0.0        0.000  -0.643548   -0.438387  42.962609  
4  -4.340968    0.0        0.000  -0.926452   -0.420000  39.924500

In [25]:
nbp_gas_monthly.columns

Index(['month', 'BASF', 'BlackburnM', 'BOCTees', 'BPGrngmouth', 'BPSaltendHP',
       'BrdgewaterP', 'Centrax', 'GooleGlass', 'HaysChem', 'ICIBillnghm',
       'ICIRuncorn', 'Immingham', 'Murco', 'PhilipsTees', 'ShellStar',
       'Winnington', 'nbp'],
      dtype='object')

In [18]:
def rystad_pivot(product, df):
    df_new = df[['month', product]].copy()
    df_new['month']=pd.to_datetime(df_new['month'])
    df_new['year']=df_new.month.dt.year
    df_new['month']=df_new.month.dt.month
    df_new = df_new.pivot(index='month', columns='year')
    df_new.reset_index(inplace=True)
    df_new['month'] = df_new['month'].apply(lambda x: calendar.month_abbr[x])
    return df_new

In [26]:
gmouth = rystad_pivot('BPGrngmouth', nbp_gas_monthly)
immingham = rystad_pivot('Immingham', nbp_gas_monthly)
BOCTees = rystad_pivot('BOCTees', nbp_gas_monthly)
BPSaltendHP = rystad_pivot('BPSaltendHP', nbp_gas_monthly)
ICIBillnghm = rystad_pivot('ICIBillnghm', nbp_gas_monthly)
ICIRuncorn = rystad_pivot('ICIRuncorn', nbp_gas_monthly)
ShellStar = rystad_pivot('ShellStar', nbp_gas_monthly)
Winnington = rystad_pivot('Winnington', nbp_gas_monthly)
BASF = rystad_pivot('BASF', nbp_gas_monthly)
nbp = rystad_pivot('nbp', nbp_gas_monthly)

In [27]:
writer = pd.ExcelWriter('uk_industrial_gas_demand_seasonal.xlsx', engine='xlsxwriter')
gmouth.to_excel(writer, sheet_name='BPGrngmouth')
immingham.to_excel(writer, sheet_name='Immingham')
BOCTees.to_excel(writer, sheet_name='BOCTees')
BPSaltendHP.to_excel(writer, sheet_name='BPSaltendHP')
ICIBillnghm.to_excel(writer, sheet_name='ICIBillnghm')
ICIRuncorn.to_excel(writer, sheet_name='ICIRuncorn')
ShellStar.to_excel(writer, sheet_name='ShellStar')
Winnington.to_excel(writer, sheet_name='Winnington')
BASF.to_excel(writer, sheet_name='BASF')
nbp.to_excel(writer, sheet_name='nbp')
writer.save()

In [24]:
# gmouth.to_excel('gmouth.xlsx')